In [1]:
from transformers import AutoModel, AutoTokenizer

In [2]:
'''
主要预训练模型:
'funnel-transformer/small'
'funnel-transformer/medium'
'funnel-transformer/large'
'funnel-transformer/xlarge'
'''
tokenizer = AutoTokenizer.from_pretrained('funnel-transformer/small')
print(tokenizer.model_input_names)  # 动态编码
print(tokenizer.all_special_tokens)

['input_ids', 'token_type_ids', 'attention_mask']
['<s>', '</s>', '<unk>', '<sep>', '<pad>', '<cls>', '<mask>']


In [3]:
model = AutoModel.from_pretrained("funnel-transformer/small")
model

FunnelModel(
  (embeddings): FunnelEmbeddings(
    (word_embeddings): Embedding(30522, 768)
    (layer_norm): LayerNorm((768,), eps=1e-09, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): FunnelEncoder(
    (attention_structure): FunnelAttentionStructure(
      (sin_dropout): Dropout(p=0.1, inplace=False)
      (cos_dropout): Dropout(p=0.1, inplace=False)
    )
    (blocks): ModuleList(
      (0): ModuleList(
        (0): FunnelLayer(
          (attention): FunnelRelMultiheadAttention(
            (hidden_dropout): Dropout(p=0.1, inplace=False)
            (attention_dropout): Dropout(p=0.1, inplace=False)
            (q_head): Linear(in_features=768, out_features=768, bias=False)
            (k_head): Linear(in_features=768, out_features=768, bias=True)
            (v_head): Linear(in_features=768, out_features=768, bias=True)
            (post_proj): Linear(in_features=768, out_features=768, bias=True)
            (layer_norm): LayerNorm((768,), 

In [4]:
model.config

FunnelConfig {
  "_name_or_path": "funnel-transformer/small",
  "activation_dropout": 0.0,
  "architectures": [
    "FunnelModel"
  ],
  "attention_dropout": 0.1,
  "attention_type": "relative_shift",
  "block_repeats": [
    1,
    1,
    1
  ],
  "block_sizes": [
    4,
    4,
    4
  ],
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "hidden_act": "gelu_new",
  "hidden_dropout": 0.1,
  "initializer_range": 0.1,
  "initializer_std": null,
  "layer_norm_eps": 1e-09,
  "max_position_embeddings": 512,
  "model_type": "funnel",
  "n_head": 12,
  "num_decoder_layers": 2,
  "pool_q_only": true,
  "pooling_type": "mean",
  "rel_attn_type": "factorized",
  "separate_cls": true,
  "transformers_version": "4.17.0",
  "truncate_seq": true,
  "type_vocab_size": 3,
  "vocab_size": 30522
}

In [5]:
for name, para in model.named_parameters():
    print(name)

embeddings.word_embeddings.weight
embeddings.layer_norm.weight
embeddings.layer_norm.bias
encoder.blocks.0.0.attention.r_w_bias
encoder.blocks.0.0.attention.r_r_bias
encoder.blocks.0.0.attention.r_kernel
encoder.blocks.0.0.attention.r_s_bias
encoder.blocks.0.0.attention.seg_embed
encoder.blocks.0.0.attention.q_head.weight
encoder.blocks.0.0.attention.k_head.weight
encoder.blocks.0.0.attention.k_head.bias
encoder.blocks.0.0.attention.v_head.weight
encoder.blocks.0.0.attention.v_head.bias
encoder.blocks.0.0.attention.post_proj.weight
encoder.blocks.0.0.attention.post_proj.bias
encoder.blocks.0.0.attention.layer_norm.weight
encoder.blocks.0.0.attention.layer_norm.bias
encoder.blocks.0.0.ffn.linear_1.weight
encoder.blocks.0.0.ffn.linear_1.bias
encoder.blocks.0.0.ffn.linear_2.weight
encoder.blocks.0.0.ffn.linear_2.bias
encoder.blocks.0.0.ffn.layer_norm.weight
encoder.blocks.0.0.ffn.layer_norm.bias
encoder.blocks.0.1.attention.r_w_bias
encoder.blocks.0.1.attention.r_r_bias
encoder.blocks.0.1

In [12]:
text = """Transformer-based models are unable to process long sequences due to their self-attention operation, which scales quadratically with the sequence length.
To address this limitation, we introduce the Longformer with an attention mechanism that scales linearly with sequence length, making it easy to process documents of thousands of tokens or longer.
Longformer’s attention mechanism is a drop-in replacement for the standard self-attention and combines a local windowed attention with a task motivated global attention.
Following prior work on long-sequence transformers, we evaluate Longformer on character-level language modeling and achieve state-of-the-art results on text8 and enwik8.
In contrast to most prior work, we also pretrain Longformer and finetune it on a variety of downstream tasks.
Our pretrained Longformer consistently outperforms RoBERTa on long document tasks and sets new state-of-the-art results on Wiki-Hop and TriviaQA."""
text = text * 4
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
print(encoded_input.keys())
print(encoded_input['input_ids'].shape)  # [1, 408]

{'input_ids': tensor([[  101, 10938,  2121,  1011,  2241,  4275,  2024,  4039,  2000,  2832,
          2146, 10071,  2349,  2000,  2037,  2969,  1011,  3086,  3169,  1010,
          2029,  9539, 17718, 23671,  3973,  2007,  1996,  5537,  3091,  1012,
          2000,  4769,  2023, 22718,  1010,  2057,  8970,  1996,  2146, 14192,
          2121,  2007,  2019,  3086,  7337,  2008,  9539,  7399,  2135,  2007,
          5537,  3091,  1010,  2437,  2009,  3733,  2000,  2832,  5491,  1997,
          5190,  1997, 19204,  2015,  2030,  2936,  1012,  2146, 14192,  2121,
          1521,  1055,  3086,  7337,  2003,  1037,  4530,  1011,  1999,  6110,
          2005,  1996,  3115,  2969,  1011,  3086,  1998, 13585,  1037,  2334,
          3332,  2098,  3086,  2007,  1037,  4708, 12774,  3795,  3086,  1012,
          2206,  3188,  2147,  2006,  2146,  1011,  5537, 19081,  1010,  2057,
         16157,  2146, 14192,  2121,  2006,  2839,  1011,  2504,  2653, 11643,
          1998,  6162,  2110,  1011,  

In [13]:
result = model(**encoded_input)
print(result.last_hidden_state)
print(result.last_hidden_state.shape)  # [1, 408, 768]

tensor([[[-0.2683, -0.2558,  0.6398,  ..., -0.1705,  1.0278, -0.4102],
         [ 0.0655, -0.2402, -0.0069,  ...,  0.5559,  0.7653,  0.1237],
         [-0.5098,  0.1188,  0.3527,  ...,  0.2985,  0.3426, -0.0684],
         ...,
         [-0.0893, -0.4039,  0.3811,  ...,  0.0233,  0.0661, -0.7399],
         [-0.2920, -0.2936,  0.3928,  ...,  0.1062,  0.7671, -1.3503],
         [-0.1974, -0.0870, -0.2085,  ...,  0.7726,  0.5676, -0.4953]]],
       grad_fn=<NativeLayerNormBackward0>)
torch.Size([1, 806, 768])
